[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [1]:
from ecdsa import SigningKey, SECP256k1

In [2]:
private_key = SigningKey.generate(curve=SECP256k1)

In [3]:
public_key = private_key.get_verifying_key()

In [7]:
message = b'I need Coffee'
signature = private_key.sign(message)

In [11]:
public_key.verify(signature, message)

True

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [20]:
class Transfer:
    
    def __init__(self, signature, public_key):
        self.signature = signature
        self.public_key = public_key
        
class ECDSACoin:
    
    def __init__(self, transfers):
        self.transfers = transfers

In [21]:
# The usual suspects ... 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

# Validating the First Transfer

In [45]:
from utils import serialize, deserialize

def issue(public_key):
    message = serialize(public_key)
    
    signature = bank_private_key.sign(message)
    transfer = Transfer(signature, public_key)
    
    coin = ECDSACoin([transfer])
    return coin

# Validating Subsequent Transfers

In [46]:
def validate(coin):
    transfer = coin.transfers
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)

In [47]:
alice_coin = issue(alice_public_key)

In [48]:
validate(alice_coin)

AttributeError: 'list' object has no attribute 'public_key'

In [49]:
bob_message = serialize(bob_public_key)
bob_signature = bob_private_key.sign(bob_message)
fake_transfer = Transfer(bob_signature, bob_public_key)
bob_fake_coin = ECDSACoin(fake_transfer)

In [50]:
from ecdsa import BadSignatureError

try: 
    validate(bob_fake_coin)
except BadSignatureError:
    print("Bad signature detected")

Bad signature detected


In [74]:
def transfer_message(previous_signature, next_owner_public_key):

    return serialize({"previous_signature":previous_signature,
                     "next_owner_public_key":next_owner_public_key
                     }
                    )

def validate(coin):
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)
    
    #check the rest
    
    previous_transfer = coin.transfers[0]
    for next_transfer in coin.transfers[1:]:
        message = transfer_message(previous_transfer.signature, next_transfer.public_key)
        previous_transfer.public_key.verify(
        next_transfer.signature, message
        )

In [75]:
def get_owner(coin):
    database = {
        serialize(bob_public_key): "Bob",
        serialize(alice_public_key): "Alice",
        serialize(bank_public_key): "Bank",
    }
    public_key = serialize(coin.transfers[-1].public_key)
    return database[public_key]

In [76]:
alice_coin = issue(alice_public_key)

print("This coin is owned by: ", get_owner(alice_coin))

This coin is owned by:  Alice


In [77]:
message = transfer_message(alice_coin.transfers[0].signature, bob_public_key)
alice_to_bob = Transfer(alice_private_key.sign(message),bob_public_key)

alice_coin.transfers.append(alice_to_bob)

In [78]:
print("This coin is owned by: ", get_owner(alice_coin))

This coin is owned by:  Bob


# Serialization

In [79]:
from utils import to_disk, from_disk

In [80]:
import os

In [81]:
filename = "coin.ecdsacoin"
print("Does coinfile exist?", os.path.isfile(filename))

Does coinfile exist? True


In [82]:
coin = issue(alice_public_key)
to_disk(coin, filename)

In [72]:
print("Does coinfile exist?", os.path.isfile(filename))

Does coinfile exist? True


# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [73]:
import ecdsacoin

coin = ecdsacoin.issue(alice_public_key)
coin.validate()

alice_to_bob = Transfer(
    signature=alice_private_key.sign(transfer_message(coin.transfers[-1].signature, bob_public_key)),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()